In [1]:
from ServiceNowData import ServiceNowHTTPXClient

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
client = ServiceNowHTTPXClient()
response = client.healthCheck() # always run to make sure the instance is awake

Request: GET https://dev85509.service-now.com/api/now/table/incident?sysparm_query=ORDERBYDESCopened_at&sysparm_limit=1
Response: 200 GET https://dev85509.service-now.com/api/now/table/incident?sysparm_query=ORDERBYDESCopened_at&sysparm_limit=1


In [3]:
# grab our major incident data
response = client.getData('incident','major_incident_state=accepted', {
    'sysparm_display_value':True,
    'sysparm_exclude_reference_link':True,
    'sysparm_fields':'number,resolved_by,short_description,description,priority,impact,urgency,closed_by,close_notes,state,category,major_incident_state'
    })

data = response.json()['result']
df = pd.DataFrame(data)
df.head(5)

Request: GET https://dev85509.service-now.com/api/now/table/incident?sysparm_query=major_incident_state%3Daccepted&sysparm_display_value=true&sysparm_exclude_reference_link=true&sysparm_fields=number%2Cresolved_by%2Cshort_description%2Cdescription%2Cpriority%2Cimpact%2Curgency%2Cclosed_by%2Cclose_notes%2Cstate%2Ccategory%2Cmajor_incident_state
Response: 200 GET https://dev85509.service-now.com/api/now/table/incident?sysparm_query=major_incident_state%3Daccepted&sysparm_display_value=true&sysparm_exclude_reference_link=true&sysparm_fields=number%2Cresolved_by%2Cshort_description%2Cdescription%2Cpriority%2Cimpact%2Curgency%2Cclosed_by%2Cclose_notes%2Cstate%2Ccategory%2Cmajor_incident_state


,number,short_description,resolved_by,urgency,impact,description,state,priority,category,major_incident_state,close_notes,closed_by
0,INC0000506,SAP HR app is working intermittently,,1 - High,1 - High,"SAP HR app either times out or loads very, ver...",In Progress,1 - Critical,Inquiry / Help,Accepted,,
1,INC0000507,SAP Controlling app down?,David Loo,1 - High,1 - High,Our entire team can't access the SAP controlli...,Resolved,1 - Critical,Software,Accepted,Identified cause as failure to connect to app ...,
2,INC0000505,Email/Exchange server is down,,1 - High,1 - High,"Email is down. Been seeing the ""unable to con...",In Progress,1 - Critical,Inquiry / Help,Accepted,,
3,INC0000530,Unable to upgrade ClipboardManager tool,,3 - Low,3 - Low,Unable to upgrade ClipboardManager tool,New,5 - Planning,Inquiry / Help,Accepted,,
4,INC0000930,Unable to downgrade Clipboard Manager tool,,3 - Low,3 - Low,Unable to downgrade Clipboard Manager tool,In Progress,5 - Planning,Inquiry / Help,Accepted,,


In [4]:
rate = 20 # average number of events per year
size = 20
# stable, shift, breakdown

z = np.random.exponential(1./rate, size=size)*365
print(f'mean: {np.mean(z)}')
print(f'max: {np.max(z)}')
print(f'min: {np.min(z)}')
print(f'std: {np.std(z)}')
df = pd.Timestamp('today').normalize() - pd.to_timedelta(24*60*60*np.cumsum(z), unit='s')
df

mean: 15.327874131264958
max: 65.93515593033428
min: 0.5343607927047794
std: 16.440591642564048


DatetimeIndex(['2023-04-20 14:19:32.856329579',
               '2023-04-03 23:12:05.401920111',
               '2023-04-01 03:38:32.437971776',
               '2023-03-17 03:46:45.869067930',
               '2023-03-16 14:57:17.096578238',
               '2023-02-22 11:46:35.181121424',
               '2022-12-18 13:19:57.708740542',
               '2022-12-11 19:12:18.940817459',
               '2022-11-17 04:40:57.877402805',
               '2022-11-03 23:45:54.941918060',
               '2022-09-30 14:20:23.451706607',
               '2022-09-20 10:21:53.819251064',
               '2022-08-11 06:03:25.304021850',
               '2022-08-09 23:03:42.709777109',
               '2022-08-03 17:25:57.310264986',
               '2022-07-31 15:51:53.298268773',
               '2022-07-25 21:07:20.578594588',
               '2022-07-24 01:02:31.603614945',
               '2022-07-19 15:34:56.205251813',
               '2022-07-18 10:37:13.501174148'],
              dtype='datetime64[ns]', f

In [5]:
from ServiceNowFaker import simulateArrivals
simulateArrivals(20,20)


DatetimeIndex(['2023-04-26 14:46:04.711184112',
               '2023-04-22 02:25:17.843435635',
               '2023-04-17 04:33:06.326981356',
               '2023-03-24 04:12:22.402238746',
               '2023-03-23 07:01:36.905766325',
               '2023-02-24 13:56:37.931085349',
               '2023-01-31 19:04:18.596797265',
               '2023-01-28 08:50:35.920789801',
               '2022-12-10 04:23:25.001710910',
               '2022-11-09 12:12:40.760502269',
               '2022-10-29 06:48:17.751850232',
               '2022-10-26 16:59:01.203025244',
               '2022-10-09 20:04:49.817834448',
               '2022-10-08 23:45:35.891787425',
               '2022-10-04 16:17:02.113674175',
               '2022-09-27 11:51:25.385775823',
               '2022-08-31 17:51:25.988978889',
               '2022-07-25 07:41:29.666499350',
               '2022-07-20 09:21:57.269767221',
               '2022-07-18 02:42:21.081033211'],
              dtype='datetime64[ns]', f